**Whether a first date will lead to a relationship**
predicting the outcome of a specific speed dating session based on the profile of two people, by implementing a recommendation system to better match people in speed dating events.This is a binary classification task. Given a data sample (information about the dating session), by predicting the probability (0-1, float) that the dating session will lead to a successful match.
The data is 191 columns as an input.
the output column (label) is match column, which is binary classification.
predict the probability that a dating session will lead to a successful match, you can use binary classification algorithms such as logistic regression, decision trees, random forests, support vector machines (SVMs), and neural networks
the challenges are Missing data: The dataset contains missing data that you will need to handle appropriately.
Imbalanced classes: The dataset might have imbalanced classes, which can make it difficult to train a model that performs well on both classes.
Feature selection: You might need to perform feature selection to identify the most important features for predicting the probability of success.
Overfitting: You might need to use techniques such as cross-validation and regularization to prevent overfitting of your model.
The impact of using this dataset to predict the probability of success for dating sessions could be significant. For example, it could help dating apps and websites to better match people based on their interests and preferences, leading to more successful matches and happier users.
However, it’s important to note that there are limitations to this dataset and the models that can be built from it. For example, the dataset only includes information about speed dating events, which might not be representative of all dating scenarios.
An ideal solution would be a model that can accurately predict the probability of success for dating sessions based on the available data in the dataset.
To achieve this, you might need to use a combination of binary classification algorithms and techniques such as feature selection, cross-validation, and regularization to build a model that performs well on both classes.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, PredefinedSplit
from xgboost.sklearn import XGBClassifier
from sklearn.datasets import make_classification
from sklearn.feature_selection import VarianceThreshold
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import roc_curve
import joblib
from sklearn_pandas import DataFrameMapper
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import make_column_transformer
from xgboost import XGBRegressor
from sklearn.preprocessing import OneHotEncoder

In [ ]:
df = pd.read_csv('/kaggle/input/cisc-873-dm-w23-a2/train.csv',index_col=0)
print(type(df))
print(df.shape)
df.head(5)

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.dtypes

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.columns[df.isnull().any()]

In [ ]:
df['match'].isnull().sum()

In [ ]:
df= df.fillna(0)

In [ ]:
df.eq(0).sum(axis=0)

In [ ]:
df.loc[:,df.dtypes == np.object].columns

In [ ]:
df.loc[:,df.dtypes == np.object]

In [ ]:
df['mn_sat'] = pd.to_numeric(df['mn_sat'], errors='coerce')
df['tuition'] = pd.to_numeric(df['tuition'], errors='coerce')
df['zipcode'] = pd.to_numeric(df['zipcode'], errors='coerce')
df['income'] = pd.to_numeric(df['income'], errors='coerce')

In [ ]:
df['field'] = df['field'].astype("category")
df['undergra'] = df['undergra'].astype("category")
df['from'] = df['from'].astype("category")
df['career'] = df['career'].astype("category")

In [ ]:
y = df['match']
X = df.drop('match', axis=1)
# print('original shape', X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# print('y_test', y_test)

In [ ]:
features_numeric = list(X_train.select_dtypes(include=['float64', 'int64']))

features_categorical = list(X_train.select_dtypes(include=['category']))

# print('numeric features:', features_numeric)
# print('categorical features:', features_categorical)

In [ ]:
np.random.seed(0)
transformer_numeric = Pipeline(
    steps=[
        ('imputer', SimpleImputer()),
        ('scaler', StandardScaler())]
)

transformer_categorical = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='constant')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', transformer_numeric, features_numeric),
        ('cat', transformer_categorical, features_categorical)
    ]
)

full_pipline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('my_classifier', 
           XGBClassifier(),
        )
    ]
)

In [ ]:
full_pipline = full_pipline.fit(X_train, y_train)
full_pipline.predict(X_test)

In [ ]:
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean'],
    'my_classifier__n_estimators': [20, 30, 40],
    'my_classifier__max_depth':[10, 20, 30],
    
}

grid_search = GridSearchCV(
    full_pipline, param_grid, cv=2, verbose=1, n_jobs=2, 
    scoring='roc_auc')

grid_search.fit(X_train, y_train)

print('best score {}'.format(grid_search.best_score_))
print('best score {}'.format(grid_search.best_params_))

In [ ]:
grid_search.predict(X_test)

In [ ]:
grid_search = GridSearchCV(
    full_pipline, param_grid, cv=5, verbose=1, n_jobs=2, 
    scoring='roc_auc')

grid_search.fit(X_train, y_train)

print('best score {}'.format(grid_search.best_score_))
print('best score {}'.format(grid_search.best_params_))

In [ ]:
random_search = RandomizedSearchCV(
    full_pipline, param_grid, cv=5, verbose=1, n_jobs=2, 
    n_iter=10,
    scoring='roc_auc')

random_search.fit(X_train, y_train)

print('best score {}'.format(random_search.best_score_))
print('best score {}'.format(random_search.best_params_))

In [ ]:
SVC_pipline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('my_svc', SVC(class_weight='balanced',probability=True))
    ]
)
bayes_search = BayesSearchCV(
    SVC_pipline,
    {
        'my_svc__C': Real(1e-6, 1e+6, prior='log-uniform'),
        'my_svc__gamma': Real(1e-6, 1e+1, prior='log-uniform'),
        'my_svc__degree': Integer(1,8),
        'my_svc__kernel': Categorical(['linear', 'poly', 'rbf']),
    },

    n_iter=3,
    random_state=0,
    verbose=1,
    cv=5,
)

bayes_search.fit(X_train, y_train)

print('best score {}'.format(bayes_search.best_score_))
print('best score {}'.format(bayes_search.best_params_))


In [ ]:
transformer_numeric = Pipeline(
    steps=[
        ('imputer', SimpleImputer()),
        ('scaler', StandardScaler())]
)

transformer_categorical = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='constant')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', transformer_numeric, features_numeric),
        ('cat', transformer_categorical, features_categorical)
    ]
)

In [ ]:
xgb_cl = xgb.XGBClassifier(eval_metric='logloss', seed=7) 
# Create XGBoost pipeline:
xgb_pipeline = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('model', xgb_cl)
])
# Evaluate the model with the use of cv:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)  #, shuffle=True with or without shuffle??
scores = cross_val_score(xgb_pipeline, X_train, y_train, cv=cv, scoring = 'roc_auc')
print("roc_auc = %f (%f)" % (scores.mean(), scores.std()))

In [ ]:
def print_results_gridsearch(gridsearch, list_param1, list_param2, name_param1, name_param2):
    # Checking the results from each run in the gridsearch: 
    means = gridsearch.cv_results_['mean_test_score']
    stds = gridsearch.cv_results_['std_test_score']
    params = gridsearch.cv_results_['params']
    print("The results from each run in the gridsearch:")
    for mean, stdev, param in zip(means, stds, params):
        print("roc_auc = %f (%f) with: %r" % (mean, stdev, param))

  # Checking the best performing model:
    print("\n")
    print("Best model: roc_auc = %f using %s" % (gridsearch.best_score_, gridsearch.best_params_))

In [ ]:
param_grid = {"model__subsample": [0.8], "model__colsample_bytree": [0.8]
              , "model__learning_rate": [0.0001, 0.001, 0.01, 0.1]
              , "model__n_estimators": range(400,500,50)
              }
#instantiate the Grid Search:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=7) 
grid_cv1 = GridSearchCV(xgb_pipeline
                        , param_grid
                        , n_jobs= -1 
                        , cv = cv
                        , scoring="roc_auc") 
# Fit
_ = grid_cv1.fit(X_train, y_train) 
# Checking the results from each run in the gridsearch:  
print_results_gridsearch(gridsearch=grid_cv1, list_param1 = param_grid["model__learning_rate"], list_param2 = param_grid["model__n_estimators"]
                         , name_param1 = 'learning_rate' , name_param2 = 'n_estimators')

In [ ]:
param_grid = {"model__subsample": [0.8], "model__colsample_bytree": [0.8], "model__learning_rate": [0.01], "model__n_estimators": [250]
              , 'model__max_depth': range(3,10,2)
              , 'model__min_child_weight': range(1,6,2)
             }
#instantiate the Grid Search:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
grid_cv2 = GridSearchCV(xgb_pipeline
                        , param_grid
                        , n_jobs=-1
                        , cv=cv
                        , scoring="roc_auc") 
# Fit
_ = grid_cv2.fit(X_train, y_train) 
# Checking the results from each run in the gridsearch:  
print_results_gridsearch(gridsearch=grid_cv2, list_param1 = param_grid["model__max_depth"], list_param2 = param_grid["model__min_child_weight"]
                         , name_param1 = 'max_depth' , name_param2 = 'min_child_weight')

In [ ]:
param_grid = {"model__learning_rate": [0.01], "model__n_estimators": [250], 'model__max_depth': [5], 'model__min_child_weight': [1]
              , 'model__subsample':[i/10.0 for i in range(4,10)]
              , 'model__colsample_bytree':[i/10.0 for i in range(4,10)] 
             }
#instantiate the Grid Search:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
grid_cv3 = GridSearchCV(xgb_pipeline
                        , param_grid
                        , n_jobs=-1
                        , cv=cv
                        , scoring="roc_auc") 
# Fit
_ = grid_cv3.fit(X_train, y_train) 
# Checking the results from each run in the gridsearch:  
print_results_gridsearch(gridsearch=grid_cv3, list_param1 = param_grid["model__subsample"], list_param2 = param_grid["model__colsample_bytree"]
                         , name_param1 = 'subsample' , name_param2 = 'colsample_bytree')

In [ ]:
param_grid = {"model__learning_rate": [0.01], "model__n_estimators": [250], 'model__max_depth': [5], 'model__min_child_weight': [1], 'model__subsample':[0.5], 'model__colsample_bytree':[0.9] 
              , "model__gamma": [i/10.0 for i in range(0,6)]
              , "model__reg_lambda": [0, 0.5, 1, 1.5, 2, 3, 4.5]
             }
              
#instantiate the Grid Search:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
grid_cv4 = GridSearchCV(xgb_pipeline
                        , param_grid
                        , n_jobs=-1
                        , cv=cv
                        , scoring="roc_auc") 
# Fit
_ = grid_cv4.fit(X_train, y_train) 
# Checking the results from each run in the gridsearch:  
print_results_gridsearch(gridsearch=grid_cv4, list_param1 = param_grid["model__gamma"], list_param2 = param_grid["model__reg_lambda"]
                         , name_param1 = 'Gamma' , name_param2 = 'Lambda')

In [ ]:
grid_cv4.best_params_

In [ ]:
xgb_pipeline.fit(X_train, y_train)
# Predict:
y_pred_meta = xgb_pipeline.predict(X_test)
y_pred_prob_meta = xgb_pipeline.predict_proba(X_test)[::,1]
# Evaluate:
print("roc_auc_score:",metrics.roc_auc_score(y_test, y_pred_meta))

In [ ]:
param_grid = {"model__subsample": [0.8], "model__colsample_bytree": [0.8]
              , "model__learning_rate": [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
              , "model__n_estimators": range(50,500,50)
              }
#instantiate the Grid Search:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=7) 
random_cv1 = RandomizedSearchCV(xgb_pipeline
                        , param_grid
                        , n_jobs= -1 
                        , cv = cv
                        , scoring="roc_auc") 
# Fit
_ = random_cv1.fit(X_train, y_train) 
# Checking the results from each run in the gridsearch:  
print_results_gridsearch(gridsearch=random_cv1, list_param1 = param_grid["model__learning_rate"], list_param2 = param_grid["model__n_estimators"]
                         , name_param1 = 'learning_rate' , name_param2 = 'n_estimators')

In [ ]:
param_grid = {"model__subsample": [0.8], "model__colsample_bytree": [0.8]
              , "model__learning_rate": [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
              , "model__n_estimators": range(50,500,50)
              }
#instantiate the Grid Search:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=7) 
bayes_cv1 = BayesSearchCV(xgb_pipeline
                        , param_grid
                        , n_jobs= -1 
                        , cv = cv
                        , scoring="roc_auc") 
# Fit
_ = bayes_cv1.fit(X_train, y_train) 
# Checking the results from each run in the gridsearch:  
print_results_gridsearch(gridsearch=bayes_cv1, list_param1 = param_grid["model__learning_rate"], list_param2 = param_grid["model__n_estimators"]
                         , name_param1 = 'learning_rate' , name_param2 = 'n_estimators')

In [ ]:
param_grid = {"model__subsample": [0.8], "model__colsample_bytree": [0.8], "model__learning_rate": [0.01], "model__n_estimators": [250]
              , 'model__max_depth': range(3,10,2)
              , 'model__min_child_weight': range(1,6,2)
             }
#instantiate the Grid Search:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
random_cv2 = RandomizedSearchCV(xgb_pipeline
                        , param_grid
                        , n_jobs=-1
                        , cv=cv
                        , scoring="roc_auc") 
# Fit
_ = random_cv2.fit(X_train, y_train) 
# Checking the results from each run in the gridsearch:  
print_results_gridsearch(gridsearch=random_cv2, list_param1 = param_grid["model__max_depth"], list_param2 = param_grid["model__min_child_weight"]
                         , name_param1 = 'max_depth' , name_param2 = 'min_child_weight')

In [ ]:
param_grid = {"model__subsample": [0.8], "model__colsample_bytree": [0.8], "model__learning_rate": [0.01], "model__n_estimators": [250]
              , 'model__max_depth': range(3,10,2)
              , 'model__min_child_weight': range(1,6,2)
             }
#instantiate the Grid Search:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
bayes_cv2 = BayesSearchCV(xgb_pipeline
                        , param_grid
                        , n_jobs=-1
                        , cv=cv
                        , scoring="roc_auc") 
# Fit
_ = bayes_cv2.fit(X_train, y_train) 
# Checking the results from each run in the gridsearch:  
print_results_gridsearch(gridsearch=bayes_cv2, list_param1 = param_grid["model__max_depth"], list_param2 = param_grid["model__min_child_weight"]
                         , name_param1 = 'max_depth' , name_param2 = 'min_child_weight')

In [ ]:
param_grid = {"model__learning_rate": [0.01], "model__n_estimators": [250], 'model__max_depth': [5], 'model__min_child_weight': [1]
              , 'model__subsample':[i/10.0 for i in range(4,10)]
              , 'model__colsample_bytree':[i/10.0 for i in range(4,10)] 
             }
#instantiate the Grid Search:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
random_cv3 = RandomizedSearchCV(xgb_pipeline
                        , param_grid
                        , n_jobs=-1
                        , cv=cv
                        , scoring="roc_auc") 
# Fit
_ = random_cv3.fit(X_train, y_train) 
# Checking the results from each run in the gridsearch:  
print_results_gridsearch(gridsearch=random_cv3, list_param1 = param_grid["model__subsample"], list_param2 = param_grid["model__colsample_bytree"]
                         , name_param1 = 'subsample' , name_param2 = 'colsample_bytree')

In [ ]:
param_grid = {"model__learning_rate": [0.01], "model__n_estimators": [250], 'model__max_depth': [5], 'model__min_child_weight': [1]
              , 'model__subsample':[i/10.0 for i in range(4,10)]
              , 'model__colsample_bytree':[i/10.0 for i in range(4,10)] 
             }
#instantiate the Grid Search:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
bayes_cv3 = BayesSearchCV(xgb_pipeline
                        , param_grid
                        , n_jobs=-1
                        , cv=cv
                        , scoring="roc_auc") 
# Fit
_ = bayes_cv3.fit(X_train, y_train) 
# Checking the results from each run in the gridsearch:  
print_results_gridsearch(gridsearch=bayes_cv3, list_param1 = param_grid["model__subsample"], list_param2 = param_grid["model__colsample_bytree"]
                         , name_param1 = 'subsample' , name_param2 = 'colsample_bytree')

In [ ]:
param_grid = {"model__learning_rate": [0.01], "model__n_estimators": [250], 'model__max_depth': [5], 'model__min_child_weight': [1], 'model__subsample':[0.5], 'model__colsample_bytree':[0.9] 
              , "model__gamma": [i/10.0 for i in range(0,6)]
              , "model__reg_lambda": [0, 0.5, 1, 1.5, 2, 3, 4.5]
             }
              
#instantiate the Grid Search:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
random_cv4 = RandomizedSearchCV(xgb_pipeline
                        , param_grid
                        , n_jobs=-1
                        , cv=cv
                        , scoring="roc_auc") 
# Fit
_ = random_cv4.fit(X_train, y_train) 
# Checking the results from each run in the gridsearch:  
print_results_gridsearch(gridsearch=random_cv4, list_param1 = param_grid["model__gamma"], list_param2 = param_grid["model__reg_lambda"]
                         , name_param1 = 'Gamma' , name_param2 = 'Lambda')

In [ ]:
param_grid = {"model__learning_rate": [0.01], "model__n_estimators": [250], 'model__max_depth': [5], 'model__min_child_weight': [1], 'model__subsample':[0.5], 'model__colsample_bytree':[0.9] 
              , "model__gamma": [i/10.0 for i in range(0,6)]
              , "model__reg_lambda": [0, 0.5, 1, 1.5, 2, 3, 4.5]
             }
              
#instantiate the Grid Search:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
bayes_cv4 = BayesSearchCV(xgb_pipeline
                        , param_grid
                        , n_jobs=-1
                        , cv=cv
                        , scoring="roc_auc") 
# Fit
_ = bayes_cv4.fit(X_train, y_train) 
# Checking the results from each run in the gridsearch:  
print_results_gridsearch(gridsearch=bayes_cv4, list_param1 = param_grid["model__gamma"], list_param2 = param_grid["model__reg_lambda"]
                         , name_param1 = 'Gamma' , name_param2 = 'Lambda')

In [ ]:
data_test = pd.read_csv('/kaggle/input/cisc-873-dm-w23-a2/test.csv',index_col=0)

In [ ]:
data_test['mn_sat'] = pd.to_numeric(data_test['mn_sat'], errors='coerce')
data_test['tuition'] = pd.to_numeric(data_test['tuition'], errors='coerce')
data_test['zipcode'] = pd.to_numeric(data_test['zipcode'], errors='coerce')
data_test['income'] = pd.to_numeric(data_test['income'], errors='coerce')

In [ ]:
data_test['field'] = data_test['field'].astype("category")
data_test['undergra'] = data_test['undergra'].astype("category")
data_test['from'] = data_test['from'].astype("category")
data_test['career'] = data_test['career'].astype("category")

In [ ]:
submission = pd.DataFrame()

submission['id'] = data_test['id']

submission['match'] = grid_search.predict_proba(data_test)[:,1]

submission.to_csv('sample_submission_walkthrough.csv', index=False)

🌈 Why a simple linear regression model (without any activation function) is not good for classification task, compared to Perceptron/Logistic regression?
A simple linear regression model without any activation function is not good for classification tasks because it cannot model the relationship between the input features and the output classes in a way that allows for non-linear decision boundaries.
In contrast, Perceptron/Logistic regression models use activation functions such as the sigmoid function to model the probability of belonging to a particular class based on the input features. This allows them to capture non-linear relationships between the input features and the output classes and to make predictions that are more accurate than those made by simple linear regression models.

🌈What's a decision tree and how it is different to a logistic regression model?
A decision tree is a type of supervised learning algorithm that is mostly used for classification tasks. It works by recursively partitioning the input space into smaller regions based on the values of the input features until it reaches a stopping criterion, such as a maximum depth or minimum number of samples per leaf node.
In contrast, logistic regression is a linear model that models the probability of belonging to a particular class based on the input features. It works by fitting a line to the data that separates the two classes and then using this line to make predictions about new data points.
The main difference between decision trees and logistic regression models is that decision trees can capture non-linear relationships between the input features and the output classes, while logistic regression models can only capture linear relationships.

🌈What's the difference between grid search and random search?
Grid search and random search are two popular methods for hyperparameter tuning in machine learning.
Grid search involves defining a grid of hyperparameters and then searching over all possible combinations of these hyperparameters to find the best model. This can be computationally expensive, especially when the number of hyperparameters is large.
Random search, on the other hand, involves randomly sampling hyperparameters from a predefined distribution and then evaluating the performance of the model with these hyperparameters. This can be much faster than grid search, especially when the number of hyperparameters is large.
While grid search is guaranteed to find the optimal set of hyperparameters (assuming that the grid is fine enough), random search can be more efficient in practice because it can explore a larger portion of the hyperparameter space in less time.

🌈What's the difference between bayesian search and random search?
Bayesian optimization is another method for hyperparameter tuning in machine learning.
Bayesian optimization uses a probabilistic model to approximate the objective function and then selects the next set of hyperparameters to evaluate based on an acquisition function that balances exploration and exploitation. This can be more efficient than random search because it can learn from previous evaluations and focus on promising regions of the hyperparameter space.
A recent study has shown that Bayesian optimization is superior to random search for machine learning hyperparameter tuning1. However, it’s worth noting that the performance of these methods can depend on the specific problem and dataset.